In [2]:
from tqdm import tqdm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
df = pd.read_csv('../data/filtered_attrs.csv')

favorable = 0
unfavorable = 1

df = df.merge(pd.read_csv('../data/bias_labels.csv'), on='url', how='inner')

privileged_groups = [{'bias':-2}]
unprivileged_groups = [{'bias':2}]

label='label'
df_output_append = df[['url', 'label']]
df.drop(columns=['url', 'source'], inplace=True)
df.dropna(inplace=True)

df = df.sample(frac=1, random_state=42)
df['label'] = df['label'].apply(lambda x: 1 if x <= 4 else 0)
df.groupby([label,'bias']).describe()

backlinks                                                  \
               count          mean           std      min        25%   
label bias                                                             
0     -2       189.0  5.605466e+06  1.157444e+07  19111.0  282383.00   
      -1       643.0  5.049366e+07  7.198760e+08  12567.0  473760.00   
       0       785.0  7.648578e+06  6.078431e+07  10490.0  260670.00   
       1       346.0  1.071284e+07  5.915935e+07  10314.0  167291.00   
       2        50.0  3.650249e+06  9.091875e+06  13961.0   98902.25   
1     -2       116.0  2.979343e+07  2.210886e+08  12316.0  211318.25   
      -1        60.0  3.429948e+07  1.085469e+08  12841.0  440475.50   
       0        17.0  1.967903e+07  4.881776e+07  16356.0  171664.00   
       1        60.0  9.225330e+06  1.690439e+07  23336.0  782770.50   
       2       348.0  4.469365e+06  1.544129e+07  10130.0  107610.25   

                                                 refpages                ...  \
                  50%          75%           max    count          mean  ...   
label bias                                                               ...   
0     -2    1250711.0   5265263.00  6.793343e+07    189.0  2.203180e+06  ...   
      -1    1859244.0   8158066.00  1.807226e+10    643.0  2.273133e+07  ...   
       0     765621.0   2398194.00  1.445707e+09    785.0  1.817495e+06  ...   
       1     725259.5   5000708.00  1.018172e+09    346.0  3.094879e+06  ...   
       2     585755.5   2563707.25  5.800030e+07     50.0  1.362081e+06  ...   
1     -2    1013283.0   4784474.75  2.348464e+09    116.0  2.001025e+07  ...   
      -1    2382958.5  15530887.25  7.284297e+08     60.0  1.627997e+07  ...   
       0    3752025.0  17177889.00  2.042046e+08     17.0  9.740917e+06  ...   
       1    3246200.0   9437863.50  8.295594e+07     60.0  3.843673e+06  ...   
       2     374743.0   1792787.25  1.545590e+08    348.0  1.595944e+06  ...   

              refips            linked_root_domains                \
                 75%        max               count          mean   
label bias                                                          
0     -2    25612.00   187798.0               189.0  12264.634921   
      -1    40362.00  1922692.0               643.0  23985.881804   
       0    20870.00   381223.0               785.0  13368.214013   
       1    26646.75   481718.0               346.0  10046.187861   
       2    12086.25    70694.0                50.0   6987.080000   
1     -2    23163.25   730914.0               116.0  12329.956897   
      -1    68460.50   614348.0                60.0  44932.383333   
       0    23201.00   141882.0                17.0  18669.294118   
       1    34229.75   199012.0                60.0   6876.283333   
       2     7833.25   319371.0               348.0   5733.505747   

                                                                     
                      std  min      25%     50%       75%       max  
label bias                                                           
0     -2     15466.784796  0.0  2204.00  6001.0  18650.00   87143.0  
      -1     43634.317359  0.0  2322.00  9314.0  33600.00  491639.0  
       0     24546.268777  0.0  1719.00  4082.0  10664.00  267165.0  
       1     18627.773151  0.0   450.50  1829.0   9125.00  155372.0  
       2     10252.849517  0.0   844.00  3062.0   8098.50   52669.0  
1     -2     19395.711611  0.0   625.75  3479.5  14814.25   87366.0  
      -1    143384.868344  0.0   573.00  5268.5  15713.50  973037.0  
       0     48579.139274  5.0   117.00  1643.0  14120.00  202141.0  
       1     14650.252811  0.0   589.00  1806.5   7039.00   83314.0  
       2     12111.714037  0.0   358.75  2032.5   5678.25  126141.0  

[10 rows x 184 columns]

In [3]:
from aif360.sklearn import metrics
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover, OptimPreproc
from aif360.algorithms.inprocessing import AdversarialDebiasing, ExponentiatedGradientReduction
import tensorflow as tf
import os
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from sklearn.ensemble import RandomForestRegressor

chosen_attribute = 'bias'
split = 0.7
binary_label_dataset = BinaryLabelDataset(
    favorable_label=favorable,
    unfavorable_label=unfavorable,
    df=df,
    label_names=[label],
    protected_attribute_names=[chosen_attribute])
orig_train, orig_test = binary_label_dataset.split([split], shuffle=False)

clf = RandomForestRegressor(n_estimators=50, random_state=0)
clf.fit(orig_train.features, orig_train.labels.ravel())
test_orig_pred = orig_test.copy()
test_orig_pred.labels = clf.predict(orig_test.features) > 0.75

original_dataset_metric = BinaryLabelDatasetMetric(test_orig_pred, 
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups)

print("{:<30} {:<15} {:<15}".format('Experiment', 'DI', 'SPD'))
print('-'*80)
print("{:<30} {:<15} {:<15}".format('Original data', f"{original_dataset_metric.disparate_impact():.4f}", f"{original_dataset_metric.statistical_parity_difference():.4f}"))


pip install 'aif360[LawSchoolGPA]'


2023-07-28 15:20:16.532243: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 15:20:16.602832: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 15:20:16.603722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 15:20:17.153381: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Experiment                     DI              SPD            
--------------------------------------------------------------------------------
Original data                  0.1811          -0.8003        


In [4]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import sys
sys.path.insert(0,'..')
import interventions.backlink_relevance_combined as intervention
import regressions.traffic_regression as regression
from importlib import reload
reload(intervention)
reload(regression)

traffic_df = pd.read_csv('../data/traffic.csv')
traffic_mod = pd.merge(traffic_df[['url', 'traffic', 'rank']], df_output_append, on='url', how='inner')
traffic_df = traffic_mod[['url', 'traffic', 'rank']]
index = orig_train.feature_names.index(chosen_attribute)

aucs = []
acc = []
f1 = []
DIs = []
spds = []
traffic_reg_scores = []
rank_reg_scores = []
for level in tqdm(np.linspace(0., 1., 11)):
    di = DisparateImpactRemover(repair_level=level, sensitive_attribute=chosen_attribute)
    train_repd = di.fit_transform(orig_train)
    test_repd = di.fit_transform(orig_test)

    X_tr = np.delete(train_repd.features, index, axis=1)
    X_te = np.delete(test_repd.features, index, axis=1)
    y_tr = train_repd.labels.ravel()
    
    clf = RandomForestRegressor(n_estimators=50, random_state=0)
    clf.fit(X_tr, y_tr)

    full_repd = di.fit_transform(binary_label_dataset)
    attrs_repd_df = pd.DataFrame(full_repd.features, columns=binary_label_dataset.feature_names)
    attrs_repd_df['url'] = df_output_append['url']
    attrs_repd_df['label'] = df_output_append['label']

    experiments = {'link scheme removal': intervention.run_combined_intervention(link_scheme_removal=True, attribute_df=attrs_repd_df)[0]}
    regression_res = regression.run_regression(traffic_df, attrs_repd_df, reg_var = 'traffic', experiments = experiments)
    traffic_reg_scores.append(regression_res)
    regression_res = regression.run_regression(traffic_df, attrs_repd_df, reg_var = 'rank', experiments = experiments)
    rank_reg_scores.append(regression_res)

    
    test_repd_pred = test_repd.copy()
    test_repd_pred.labels = clf.predict(X_te) > 0.75
    aucs.append(roc_auc_score(test_repd.labels.ravel(), test_repd_pred.labels))
    acc.append(accuracy_score(test_repd.labels.ravel(), test_repd_pred.labels))
    f1.append(f1_score(test_repd.labels.ravel(), test_repd_pred.labels, average='macro'))

    cm = BinaryLabelDatasetMetric(test_repd_pred, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
    DIs.append(cm.disparate_impact())
    spds.append(cm.statistical_parity_difference())


  9%|▉         | 1/11 [00:01<00:14,  1.48s/it]

Num link schemes:  91
Source domains with scores:  0


 18%|█▊        | 2/11 [00:02<00:13,  1.47s/it]

Num link schemes:  91
Source domains with scores:  0


 27%|██▋       | 3/11 [00:04<00:11,  1.46s/it]

...

Num link schemes:  91
Source domains with scores:  0


 91%|█████████ | 10/11 [00:14<00:01,  1.46s/it]

Num link schemes:  91
Source domains with scores:  0


100%|██████████| 11/11 [00:15<00:00,  1.43s/it]

Num link schemes:  91
Source domains with scores:  0


In [14]:
import matplotlib.pyplot as plt
import numpy as np
x_axis_data = np.linspace(0, 1, 11)
fig, ax2 = plt.subplots()
ax2.set_xlabel('Repair Level r') 
ax2.plot(x_axis_data, [x['metric'][0] for x in traffic_reg_scores], 'C2-', label='Traffic RIS Scores')
ax2.plot(x_axis_data, [x['metric'][0] for x in rank_reg_scores], 'C3-', label='Rank RIS Scores')
ax2.set_ylabel('RIS Score')
ax2.tick_params(axis='y')
ax2.set_ylim(1,3)
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines2, labels2, loc='upper right')

plt.title('Debiasing (r) is not at the cost of Intervention Quality (RIS)')
plt.tight_layout()
plt.savefig('../results/ris.png', dpi=300)

In [18]:
import matplotlib.pyplot as plt
import numpy as np

x_axis_data = np.linspace(0, 1, 11)
fig, ax1 = plt.subplots()
ax1.plot(x_axis_data, DIs, 'C0-', label='DI Scores')
ax1.set_xlabel('Repair Level r') 
ax1.tick_params(axis='y')
ax1.plot(x_axis_data, f1, 'C1-', label='F1 Scores')
ax1.set_ylim(0,1)
lines, labels = ax1.get_legend_handles_labels()
ax1.legend(lines, labels, loc='upper right')

plt.title('Debiasing (r) is at the cost of Classification Performance (F1)')
plt.tight_layout()
plt.savefig('../results/f1.png', dpi=300)
plt.show()


In [6]:
di = DisparateImpactRemover(repair_level=1, sensitive_attribute=chosen_attribute)
transformed_dataset = di.fit_transform(binary_label_dataset)

transformed_dataset_metric = BinaryLabelDatasetMetric(transformed_dataset, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
print("{:<30} {:<15} {:<15}".format('Transformed data', f"{transformed_dataset_metric.disparate_impact():.4f}",  f"{transformed_dataset_metric.statistical_parity_difference():.4f}"))

Transformed data               0.2027          -0.4940        


In [7]:
# create df out of numpy array
df_output = pd.DataFrame(transformed_dataset.features, columns=binary_label_dataset.feature_names)
df_output['url'] = df_output_append['url']
df_output['label'] = df_output_append['label']
df_output.to_csv('results/transformed_features.csv', index=False)

Index(['backlinks', 'refpages', 'pages', 'valid_pages', 'text', 'image',
       'nofollow', 'ugc', 'sponsored', 'dofollow', 'redirect', 'canonical',
       'gov', 'edu', 'rss', 'alternate', 'html_pages', 'links_internal',
       'links_external', 'refdomains', 'refclass_c', 'refips',
       'linked_root_domains', 'bias', 'url', 'label'],
      dtype='object')